In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys
import random
from datetime import datetime, timedelta

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 10.0
MAX_DESTINATION_DEVIATION_KM = 5.0
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0

# Database Collection Names
RIDER_COLLECTION_NAME = 'riders'
PROPOSAL_COLLECTION_NAME = 'driver_proposals'
PUBLIC_RIDE_REQUESTS_COLLECTION = 'public_ride_requests'

# Ride Request States (Passenger DB)
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ARRIVED_AT_PICKUP = 'arrived_at_pickup'
STATUS_PICKED_UP = 'picked_up'
STATUS_ON_WAY = 'on_way'
STATUS_COMPLETED = 'completed'
STATUS_CANCELLED = 'cancelled'

# Rider Status States (Driver DB)
RIDER_AVAILABLE = 'available'
RIDER_ON_TRIP = 'on_trip'
RIDER_OFFLINE = 'offline'

# Proposal Status States - FIXED: Use 'pending_acceptance' to match rider app
PROPOSAL_PENDING = 'pending_acceptance'  # CHANGED from 'pending'
PROPOSAL_ACCEPTED = 'accepted'
PROPOSAL_REJECTED = 'rejected'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance between two coordinates in kilometers."""
    R = 6371
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """Calculates a dynamic pickup point if within threshold."""
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    """Checks if passenger destination is near driver destination."""
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False

def calculate_match_score(distance, rating, passenger_preference, rider_vehicle_type, total_rides):
    """Calculate match score based on multiple factors (same as Flutter app)."""
    distance_score = 1.0 - (distance / 10000)  # Normalize to 0-1
    rating_score = rating / 5.0  # Normalize to 0-1
    experience_score = min(total_rides / 100.0, 1.0)  # Cap at 1.0
    preference_score = 1.0 if (passenger_preference == 'Any' or rider_vehicle_type == passenger_preference) else 0.5
    
    return (distance_score * 0.4) + (rating_score * 0.3) + (experience_score * 0.2) + (preference_score * 0.1)

def calculate_priority_level(match_score):
    """Calculate priority level based on match score."""
    if match_score >= 0.8:
        return 1
    elif match_score >= 0.6:
        return 2
    else:
        return 3

def calculate_fare(distance_km, ride_type):
    """Calculate fare based on distance and ride type (same as Flutter app)."""
    base_fare = 30.0
    rate_per_km = 8.0
    
    # Different pricing for different ride types
    if ride_type == 'Premium':
        base_fare = 50.0
        rate_per_km = 12.0
    elif ride_type == 'SUV':
        base_fare = 70.0
        rate_per_km = 15.0
    elif ride_type == 'Electric':
        base_fare = 35.0
        rate_per_km = 7.0
    
    return base_fare + (distance_km * rate_per_km)

def generate_otp():
    """Generate 4-digit OTP (same as Flutter app)."""
    return str(random.randint(1000, 9999))

# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """Executed when rider ACCEPTS the proposed ride."""
    ride_id = proposal_data['original_request_id']
    rider_id = proposal_data['riderUid']
    
    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Rider {rider_id}.")

    try:
        # 1. Update Public Ride Request to ACCEPTED
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED,
            'riderUid': rider_id,
            'riderName': proposal_data.get('riderName', 'Rider'),
            'riderPhone': proposal_data.get('riderPhone', 'N/A'),
            'riderRating': proposal_data.get('riderRating', 5.0),
            'vehicleNumber': proposal_data.get('vehicleNumber'),
            'vehicleModel': proposal_data.get('vehicleModel'),
            'vehicleColor': proposal_data.get('vehicleColor'),
            'vehicleType': proposal_data.get('vehicleType'),
            'totalRides': proposal_data.get('totalRides', 0),
            'matched_at': firestore.SERVER_TIMESTAMP,
        })
        print(f"  [DB UPDATE] Public ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Rider status to on_trip
        rider_ref = ddb.collection(RIDER_COLLECTION_NAME).document(rider_id)
        rider_ref.update({
            'status': RIDER_ON_TRIP,
            'current_passenger_id': proposal_data.get('passengerId'),
            'current_ride_id': ride_id,
            'lastActive': firestore.SERVER_TIMESTAMP(),
        })
        print(f"  [DB UPDATE] Rider {rider_id} status changed to '{RIDER_ON_TRIP}'.")
        
        # 3. Update ALL proposals for this ride to accepted
        proposal_query = ddb.collection(PROPOSAL_COLLECTION_NAME).where('original_request_id', '==', ride_id)
        proposals = proposal_query.get()
        for proposal in proposals:
            proposal.reference.update({
                'status': PROPOSAL_ACCEPTED,
                'acceptedTimestamp': firestore.SERVER_TIMESTAMP(),
            })
            print(f"  [DB UPDATE] Proposal {proposal.id} updated to 'accepted'.")
        
        print(f"✅ Ride {ride_id} successfully matched with Rider {rider_id}")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """Executed when rider REJECTS the proposed ride."""
    ride_id = proposal_data['original_request_id']
    proposal_id = proposal_data.get('proposal_id')
    
    print(f"\n[REJECTION] Rider rejected Ride {ride_id}. Cleaning up proposal.")
    
    try:
        # Update specific proposal status to rejected
        if proposal_id:
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(proposal_id)
            proposal_ref.update({
                'status': PROPOSAL_REJECTED,
                'rejectedTimestamp': firestore.SERVER_TIMESTAMP()
            })
            print(f"  [DB UPDATE] Proposal {proposal_id} updated to 'rejected'.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_ride_status_update(pdb, ddb, proposal_data, new_status):
    """Handle ride status updates from rider app."""
    ride_id = proposal_data['original_request_id']
    
    try:
        # Update public ride request status
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        
        update_data = {
            'status': new_status,
            'lastUpdated': firestore.SERVER_TIMESTAMP()
        }
        
        # Add timestamp based on status
        if new_status == STATUS_ARRIVED_AT_PICKUP:
            update_data['arrivalTimestamp'] = firestore.SERVER_TIMESTAMP()
        elif new_status == STATUS_PICKED_UP:
            update_data['pickupTimestamp'] = firestore.SERVER_TIMESTAMP()
            update_data['otpVerified'] = True
        elif new_status == STATUS_COMPLETED:
            update_data['completionTimestamp'] = firestore.SERVER_TIMESTAMP()
        elif new_status == STATUS_CANCELLED:
            update_data['cancellationTimestamp'] = firestore.SERVER_TIMESTAMP()
        
        ride_ref.update(update_data)
        print(f"  [DB UPDATE] Ride {ride_id} status updated to '{new_status}'.")
        
    except Exception as e:
        print(f"[ERROR] Failed to update ride status for {ride_id}: {e}")

# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """Finds suitable riders and creates proposals."""
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Extract passenger data from public_ride_requests
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    passenger_id = request_data.get('passengerId')
    passenger_name = request_data.get('passengerName', 'Passenger')
    passenger_phone = request_data.get('passengerPhone', 'N/A')
    pickup_address = request_data.get('pickupAddress', 'Pickup Location')
    destination_address = request_data.get('destinationAddress', 'Destination Location')
    ride_type = request_data.get('rideType', 'Standard')
    vehicle_preference = request_data.get('vehiclePreference', 'Any')
    passenger_rating = request_data.get('passengerRating', 5.0)
    special_requests = request_data.get('specialRequests', 'None')
    luggage_count = request_data.get('luggageCount', 0)
    passenger_count = request_data.get('passengerCount', 1)
    payment_method = request_data.get('paymentMethod', 'Cash')

    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for riders for Ride ID: {ride_id}")
    print(f"Passenger: {passenger_name} | Vehicle Preference: {vehicle_preference}")
    
    # Query for available riders
    available_riders_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_AVAILABLE)
    potential_riders = available_riders_query.stream()
    rider_list = list(potential_riders)
    
    total_riders_checked = 0
    suitable_riders = []

    for rider_doc in rider_list:
        rider_data = rider_doc.to_dict()
        rider_id = rider_doc.id
        
        # Rider data from your rider app structure
        rider_current_loc = rider_data.get('currentLocation')
        rider_dest_loc = rider_data.get('currentRouteEnd')  # Rider's destination
        rider_name = rider_data.get('riderName', rider_data.get('name', 'Rider'))
        rider_phone = rider_data.get('phone', 'N/A')
        rider_vehicle_type = rider_data.get('vehicleType', '')
        rider_rating = rider_data.get('rating', 5.0)
        total_rides = rider_data.get('total_rides', 0)
        vehicle_number = rider_data.get('vehicle_number')
        vehicle_model = rider_data.get('vehicle_model')
        vehicle_color = rider_data.get('vehicle_color')

        if not rider_current_loc:
            continue

        total_riders_checked += 1
        
        # Calculate distance to pickup
        distance_to_pickup = calculate_distance(
            rider_current_loc.latitude, rider_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        # Check if rider is within maximum match distance (10km as per Flutter app)
        if distance_to_pickup > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {rider_id}: Too far ({distance_to_pickup:.2f} km).")
            continue

        # Check destination overlap if rider has a destination
        destination_overlap = True
        if rider_dest_loc:
            destination_overlap = check_destination_overlap(rider_dest_loc, passenger_dest_loc)
        
        if not destination_overlap:
            print(f"  Skipping {rider_id}: Destination doesn't overlap.")
            continue

        # Check Vehicle Type Match
        if vehicle_preference != 'Any' and vehicle_preference.lower() not in rider_vehicle_type.lower():
            print(f"  Skipping {rider_id}: Vehicle preference mismatch.")
            continue

        # Calculate match score (same algorithm as Flutter app)
        match_score = calculate_match_score(
            distance_to_pickup * 1000,  # Convert to meters for consistency
            rider_rating,
            vehicle_preference,
            rider_vehicle_type,
            total_rides
        )

        # Only consider riders with good match score
        if match_score >= 0.6:
            suitable_riders.append({
                'rider_id': rider_id,
                'rider_data': rider_data,
                'rider_loc': rider_current_loc,
                'rider_dest': rider_dest_loc,
                'rider_name': rider_name,
                'rider_phone': rider_phone,
                'vehicle_type': rider_vehicle_type,
                'rating': rider_rating,
                'total_rides': total_rides,
                'vehicle_number': vehicle_number,
                'vehicle_model': vehicle_model,
                'vehicle_color': vehicle_color,
                'distance_to_pickup': distance_to_pickup,
                'match_score': match_score,
                'priority_level': calculate_priority_level(match_score)
            })
            print(f"  ✅ Suitable rider found: {rider_name} (Score: {match_score:.2f}, Distance: {distance_to_pickup:.2f} km)")

    print(f"  Total {total_riders_checked} riders checked, {len(suitable_riders)} suitable riders found.")

    # Sort riders by match score (highest first)
    suitable_riders.sort(key=lambda x: x['match_score'], reverse=True)
    
    # Create proposals for top suitable riders
    for rider_info in suitable_riders[:5]:  # Limit to top 5 riders
        _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info)

def _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info):
    """Create a proposal document for a specific rider."""
    try:
        # Calculate fare
        passenger_pickup_loc = request_data.get('pickupLocation')
        passenger_dest_loc = request_data.get('destinationLocation')
        
        ride_distance = calculate_distance(
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude,
            passenger_dest_loc.latitude, passenger_dest_loc.longitude
        )
        
        fare_amount = calculate_fare(ride_distance, request_data.get('rideType', 'Standard'))
        
        # Generate OTP
        otp = generate_otp()
        
        # Create proposal document with ALL fields that rider app expects
        proposal_data = {
            'original_request_id': ride_id,
            'riderUid': rider_info['rider_id'],  # This is CRITICAL for rider app filtering
            'riderName': rider_info['rider_name'],
            'riderPhone': rider_info['rider_phone'],
            'riderRating': rider_info['rating'],
            'vehicleNumber': rider_info['vehicle_number'],
            'vehicleModel': rider_info['vehicle_model'],
            'vehicleColor': rider_info['vehicle_color'],
            'vehicleType': rider_info['vehicle_type'],
            'totalRides': rider_info['total_rides'],
            'passengerId': request_data.get('passengerId'),
            'passengerName': request_data.get('passengerName', 'Passenger'),
            'passengerPhone': request_data.get('passengerPhone', 'N/A'),
            'pickupLocation': request_data.get('pickupLocation'),
            'destinationLocation': request_data.get('destinationLocation'),
            'pickupAddress': request_data.get('pickupAddress', 'Pickup Location'),
            'destinationAddress': request_data.get('destinationAddress', 'Destination Location'),
            'fareAmount': fare_amount,
            'paymentMethod': request_data.get('paymentMethod', 'Cash'),
            'rideType': request_data.get('rideType', 'Standard'),
            'passengerRating': request_data.get('passengerRating', 5.0),
            'estimatedDistance': f"{ride_distance:.1f} km",
            'estimatedDuration': f"{(ride_distance / 40 * 60):.0f} min",  # Assuming 40km/h average
            'specialRequests': request_data.get('specialRequests', 'None'),
            'vehiclePreference': request_data.get('vehiclePreference', 'Any'),
            'luggageCount': request_data.get('luggageCount', 0),
            'passengerCount': request_data.get('passengerCount', 1),
            'status': PROPOSAL_PENDING,  # This is now 'pending_acceptance'
            'requestTimestamp': firestore.SERVER_TIMESTAMP,
            'distanceToPickup': rider_info['distance_to_pickup'] * 1000,  # Convert to meters
            'match_score': rider_info['match_score'],
            'priority_level': rider_info['priority_level'],
            'otp': otp,
            'otpVerified': False,
            # Additional fields that rider app might expect
            'createdAt': firestore.SERVER_TIMESTAMP,
            'proposalExpiresAt': firestore.SERVER_TIMESTAMP,  # Add expiration
        }
        
        # Add proposal to driver_proposals collection
        proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document()
        proposal_ref.set(proposal_data)
        
        print(f"  📨 Proposal sent to {rider_info['rider_name']} (ID: {proposal_ref.id})")
        print(f"     Match Score: {rider_info['match_score']:.2f}, Fare: ₹{fare_amount:.2f}, OTP: {otp}")
        print(f"     Status: {PROPOSAL_PENDING}, Rider UID: {rider_info['rider_id']}")
        
    except Exception as e:
        print(f"[ERROR] Failed to create proposal for rider {rider_info['rider_id']}: {e}")

# --- Snapshot Listeners ---

def on_new_ride_request_snapshot(col_snapshot, changes, read_time):
    """Listener for new ride requests in public_ride_requests."""
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            # Only process new pending requests
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New PENDING ride request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)

def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """Listener for rider responses to proposals and status updates."""
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            previous_data = getattr(change.document, '_previous', {})
            
            current_status = data.get('status')
            previous_status = previous_data.get('status') if previous_data else None
            
            print(f"[PROPOSAL UPDATE] Proposal {change.document.id}: {previous_status} -> {current_status}")
            
            # Handle rider acceptance/rejection
            if (current_status != previous_status and 
                current_status in [PROPOSAL_ACCEPTED, PROPOSAL_REJECTED]):
                
                if current_status == PROPOSAL_ACCEPTED:
                    print(f"[EVENT] Rider accepted proposal: {change.document.id}")
                    # Add proposal ID to data for cleanup
                    data['proposal_id'] = change.document.id
                    finalize_match(db_passenger, db_driver, data)
                elif current_status == PROPOSAL_REJECTED:
                    print(f"[EVENT] Rider rejected proposal: {change.document.id}")
                    # Add proposal ID to data for cleanup
                    data['proposal_id'] = change.document.id
                    handle_rejection(db_passenger, db_driver, data)
            
            # Handle ride status updates from rider app
            elif current_status in [STATUS_ARRIVED_AT_PICKUP, STATUS_PICKED_UP, STATUS_ON_WAY, STATUS_COMPLETED, STATUS_CANCELLED]:
                if current_status != previous_status:
                    print(f"[RIDE STATUS] Ride status changed to: {current_status}")
                    handle_ride_status_update(db_passenger, db_driver, data, current_status)

def on_rider_status_snapshot(col_snapshot, changes, read_time):
    """Listener for rider status changes to track online/offline riders."""
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            rider_id = change.document.id
            status = data.get('status', 'offline')
            location = data.get('currentLocation')
            
            if status == RIDER_AVAILABLE and location:
                print(f"[RIDER STATUS] Rider {rider_id} is available at ({location.latitude:.4f}, {location.longitude:.4f})")
            elif status == RIDER_ON_TRIP:
                print(f"[RIDER STATUS] Rider {rider_id} is on trip")
            elif status == RIDER_OFFLINE:
                print(f"[RIDER STATUS] Rider {rider_id} went offline")

def cleanup_old_proposals():
    """Clean up old proposals that are no longer relevant."""
    try:
        # Delete proposals older than 1 hour
        one_hour_ago = datetime.now() - timedelta(hours=1)
        
        proposals_query = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        old_proposals = proposals_query.get()
        
        deleted_count = 0
        for proposal in old_proposals:
            proposal_data = proposal.to_dict()
            proposal_time = proposal_data.get('requestTimestamp')
            
            # Skip if no timestamp
            if not proposal_time:
                continue
                
            # Convert Firestore timestamp to datetime
            if hasattr(proposal_time, 'timestamp'):
                proposal_datetime = datetime.fromtimestamp(proposal_time.timestamp())
            else:
                continue
                
            if proposal_datetime < one_hour_ago:
                if proposal_data.get('status') in [PROPOSAL_PENDING, PROPOSAL_REJECTED]:
                    proposal.reference.delete()
                    deleted_count += 1
                    print(f"🧹 Cleaned up old proposal: {proposal.id}")
        
        if deleted_count > 0:
            print(f"🧹 Cleaned up {deleted_count} old proposals")
                
    except Exception as e:
        print(f"[CLEANUP ERROR] Failed to clean up old proposals: {e}")

# --- Main Execution ---

if __name__ == '__main__':
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"🚗 RideHail Pro Matching Server Started")
    print(f"📊 Configuration:")
    print(f"   - Max Match Distance: {MAX_MATCH_DISTANCE_KM} km")
    print(f"   - Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km")
    print(f"   - Dynamic Pickup Threshold: {DYNAMIC_PICKUP_THRESHOLD_KM} km")
    print(f"   - Proposal Status: {PROPOSAL_PENDING}")  # Show the corrected status
    print(f"   - Listening on collections: public_ride_requests, driver_proposals, riders")
    
    try:
        # Listener 1: Watch for new ride requests in public_ride_requests
        query_ride_requests = db_passenger.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).where('status', '==', STATUS_PENDING)
        query_ride_requests.on_snapshot(on_new_ride_request_snapshot)
        
        # Listener 2: Watch for rider responses in driver_proposals
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        # Listener 3: Watch for rider status changes
        query_riders = db_driver.collection(RIDER_COLLECTION_NAME)
        query_riders.on_snapshot(on_rider_status_snapshot)
        
        print("-" * 60)
        print("✅ Three listeners started successfully:")
        print("   1. New Ride Requests (public_ride_requests)")
        print("   2. Rider Responses (driver_proposals)") 
        print("   3. Rider Status Changes (riders)")
        print(f"📨 Proposal Status: '{PROPOSAL_PENDING}' (Fixed to match rider app)")
        print("Press Ctrl+C to stop the server.")
        print("-" * 60)

        # Start cleanup scheduler
        import threading
        def schedule_cleanup():
            while True:
                time.sleep(3600)  # Run every hour
                cleanup_old_proposals()
        
        cleanup_thread = threading.Thread(target=schedule_cleanup, daemon=True)
        cleanup_thread.start()
        
        # Keep the main thread alive
        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("\n🛑 RideHail Pro Matching Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)

Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
🚗 RideHail Pro Matching Server Started
📊 Configuration:
   - Max Match Distance: 10.0 km
   - Max Destination Deviation: 5.0 km
   - Dynamic Pickup Threshold: 2.0 km
   - Proposal Status: pending_acceptance
   - Listening on collections: public_ride_requests, driver_proposals, riders
------------------------------------------------------------
✅ Three listeners started successfully:
   1. New Ride Requests (public_ride_requests)
   2. Rider Responses (driver_proposals)
   3. Rider Status Changes (riders)
📨 Proposal Status: 'pending_acceptance' (Fixed to match rider app)
Press Ctrl+C to stop the server.
------------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[EVENT] New PENDING ride request detected: BKhF7HGtdwchPjJgjajO

[MATCHING] Searching for riders for Ride ID: BKhF7HGtdwchPjJgjajO
Passenger: chandan68gowda | Vehicle Preference: Any
[RIDER STATUS] Rider KHoVWhQqYOZhRRki0VK9 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5432)


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys
import random
from datetime import datetime, timedelta

# --- Configuration ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 10.0
MAX_DESTINATION_DEVIATION_KM = 5.0
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0

# Database Collection Names
RIDER_COLLECTION_NAME = 'riders'
PROPOSAL_COLLECTION_NAME = 'driver_proposals'
PUBLIC_RIDE_REQUESTS_COLLECTION = 'public_ride_requests'

# Ride Request States (Passenger DB)
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ARRIVED_AT_PICKUP = 'arrived_at_pickup'
STATUS_PICKED_UP = 'picked_up'
STATUS_ON_WAY = 'on_way'
STATUS_COMPLETED = 'completed'
STATUS_CANCELLED = 'cancelled'

# Rider Status States (Driver DB)
RIDER_AVAILABLE = 'available'
RIDER_ON_TRIP = 'on_trip'
RIDER_OFFLINE = 'offline'

# Proposal Status States - FIXED: Use 'pending_acceptance' to match rider app
PROPOSAL_PENDING = 'pending_acceptance'  # CHANGED from 'pending'
PROPOSAL_ACCEPTED = 'accepted'
PROPOSAL_REJECTED = 'rejected'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance between two coordinates in kilometers."""
    R = 6371
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """Calculates a dynamic pickup point if within threshold."""
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    """Checks if passenger destination is near driver destination."""
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False

def calculate_match_score(distance, rating, passenger_preference, rider_vehicle_type, total_rides):
    """Calculate match score based on multiple factors (same as Flutter app)."""
    distance_score = 1.0 - (distance / 10000)  # Normalize to 0-1
    rating_score = rating / 5.0  # Normalize to 0-1
    experience_score = min(total_rides / 100.0, 1.0)  # Cap at 1.0
    preference_score = 1.0 if (passenger_preference == 'Any' or rider_vehicle_type == passenger_preference) else 0.5
    
    return (distance_score * 0.4) + (rating_score * 0.3) + (experience_score * 0.2) + (preference_score * 0.1)

def calculate_priority_level(match_score):
    """Calculate priority level based on match score."""
    if match_score >= 0.8:
        return 1
    elif match_score >= 0.6:
        return 2
    else:
        return 3

def calculate_fare(distance_km, ride_type):
    """Calculate fare based on distance and ride type (same as Flutter app)."""
    base_fare = 30.0
    rate_per_km = 8.0
    
    # Different pricing for different ride types
    if ride_type == 'Premium':
        base_fare = 50.0
        rate_per_km = 12.0
    elif ride_type == 'SUV':
        base_fare = 70.0
        rate_per_km = 15.0
    elif ride_type == 'Electric':
        base_fare = 35.0
        rate_per_km = 7.0
    
    return base_fare + (distance_km * rate_per_km)

def generate_otp():
    """Generate 4-digit OTP (same as Flutter app)."""
    return str(random.randint(1000, 9999))

# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """Executed when rider ACCEPTS the proposed ride."""
    ride_id = proposal_data['original_request_id']
    rider_id = proposal_data['riderUid']
    
    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Rider {rider_id}.")

    try:
        # 1. Update Public Ride Request to ACCEPTED
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED,
            'riderUid': rider_id,
            'riderName': proposal_data.get('riderName', 'Rider'),
            'riderPhone': proposal_data.get('riderPhone', 'N/A'),
            'riderRating': proposal_data.get('riderRating', 5.0),
            'vehicleNumber': proposal_data.get('vehicleNumber'),
            'vehicleModel': proposal_data.get('vehicleModel'),
            'vehicleColor': proposal_data.get('vehicleColor'),
            'vehicleType': proposal_data.get('vehicleType'),
            'totalRides': proposal_data.get('totalRides', 0),
            'matched_at': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
        })
        print(f"  [DB UPDATE] Public ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Rider status to on_trip
        rider_ref = ddb.collection(RIDER_COLLECTION_NAME).document(rider_id)
        rider_ref.update({
            'status': RIDER_ON_TRIP,
            'current_passenger_id': proposal_data.get('passengerId'),
            'current_ride_id': ride_id,
            'lastActive': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
        })
        print(f"  [DB UPDATE] Rider {rider_id} status changed to '{RIDER_ON_TRIP}'.")
        
        # 3. Update ALL proposals for this ride to accepted
        proposal_query = ddb.collection(PROPOSAL_COLLECTION_NAME).where('original_request_id', '==', ride_id)
        proposals = proposal_query.get()
        for proposal in proposals:
            proposal.reference.update({
                'status': PROPOSAL_ACCEPTED,
                'acceptedTimestamp': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
            })
            print(f"  [DB UPDATE] Proposal {proposal.id} updated to 'accepted'.")
        
        print(f"✅ Ride {ride_id} successfully matched with Rider {rider_id}")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """Executed when rider REJECTS the proposed ride."""
    ride_id = proposal_data['original_request_id']
    proposal_id = proposal_data.get('proposal_id')
    
    print(f"\n[REJECTION] Rider rejected Ride {ride_id}. Cleaning up proposal.")
    
    try:
        # Update specific proposal status to rejected
        if proposal_id:
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(proposal_id)
            proposal_ref.update({
                'status': PROPOSAL_REJECTED,
                'rejectedTimestamp': firestore.SERVER_TIMESTAMP  # FIXED: No parentheses
            })
            print(f"  [DB UPDATE] Proposal {proposal_id} updated to 'rejected'.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_ride_status_update(pdb, ddb, proposal_data, new_status):
    """Handle ride status updates from rider app."""
    ride_id = proposal_data['original_request_id']
    
    try:
        # Update public ride request status
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        
        update_data = {
            'status': new_status,
            'lastUpdated': firestore.SERVER_TIMESTAMP  # FIXED: No parentheses
        }
        
        # Add timestamp based on status - FIXED: No parentheses on SERVER_TIMESTAMP
        if new_status == STATUS_ARRIVED_AT_PICKUP:
            update_data['arrivalTimestamp'] = firestore.SERVER_TIMESTAMP
        elif new_status == STATUS_PICKED_UP:
            update_data['pickupTimestamp'] = firestore.SERVER_TIMESTAMP
            update_data['otpVerified'] = True
        elif new_status == STATUS_COMPLETED:
            update_data['completionTimestamp'] = firestore.SERVER_TIMESTAMP
        elif new_status == STATUS_CANCELLED:
            update_data['cancellationTimestamp'] = firestore.SERVER_TIMESTAMP
        
        ride_ref.update(update_data)
        print(f"  [DB UPDATE] Ride {ride_id} status updated to '{new_status}'.")
        
    except Exception as e:
        print(f"[ERROR] Failed to update ride status for {ride_id}: {e}")

# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """Finds suitable riders and creates proposals."""
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    # Extract passenger data from public_ride_requests
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    passenger_id = request_data.get('passengerId')
    passenger_name = request_data.get('passengerName', 'Passenger')
    passenger_phone = request_data.get('passengerPhone', 'N/A')
    pickup_address = request_data.get('pickupAddress', 'Pickup Location')
    destination_address = request_data.get('destinationAddress', 'Destination Location')
    ride_type = request_data.get('rideType', 'Standard')
    vehicle_preference = request_data.get('vehiclePreference', 'Any')
    passenger_rating = request_data.get('passengerRating', 5.0)
    special_requests = request_data.get('specialRequests', 'None')
    luggage_count = request_data.get('luggageCount', 0)
    passenger_count = request_data.get('passengerCount', 1)
    payment_method = request_data.get('paymentMethod', 'Cash')

    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for riders for Ride ID: {ride_id}")
    print(f"Passenger: {passenger_name} | Vehicle Preference: {vehicle_preference}")
    
    # Query for available riders
    available_riders_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_AVAILABLE)
    potential_riders = available_riders_query.stream()
    rider_list = list(potential_riders)
    
    total_riders_checked = 0
    suitable_riders = []

    for rider_doc in rider_list:
        rider_data = rider_doc.to_dict()
        rider_id = rider_doc.id
        
        # Rider data from your rider app structure
        rider_current_loc = rider_data.get('currentLocation')
        rider_dest_loc = rider_data.get('currentRouteEnd')  # Rider's destination
        rider_name = rider_data.get('riderName', rider_data.get('name', 'Rider'))
        rider_phone = rider_data.get('phone', 'N/A')
        rider_vehicle_type = rider_data.get('vehicleType', '')
        rider_rating = rider_data.get('rating', 5.0)
        total_rides = rider_data.get('total_rides', 0)
        vehicle_number = rider_data.get('vehicle_number')
        vehicle_model = rider_data.get('vehicle_model')
        vehicle_color = rider_data.get('vehicle_color')

        if not rider_current_loc:
            continue

        total_riders_checked += 1
        
        # Calculate distance to pickup
        distance_to_pickup = calculate_distance(
            rider_current_loc.latitude, rider_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        # Check if rider is within maximum match distance (10km as per Flutter app)
        if distance_to_pickup > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {rider_id}: Too far ({distance_to_pickup:.2f} km).")
            continue

        # Check destination overlap if rider has a destination
        destination_overlap = True
        if rider_dest_loc:
            destination_overlap = check_destination_overlap(rider_dest_loc, passenger_dest_loc)
        
        if not destination_overlap:
            print(f"  Skipping {rider_id}: Destination doesn't overlap.")
            continue

        # Check Vehicle Type Match
        if vehicle_preference != 'Any' and vehicle_preference.lower() not in rider_vehicle_type.lower():
            print(f"  Skipping {rider_id}: Vehicle preference mismatch.")
            continue

        # Calculate match score (same algorithm as Flutter app)
        match_score = calculate_match_score(
            distance_to_pickup * 1000,  # Convert to meters for consistency
            rider_rating,
            vehicle_preference,
            rider_vehicle_type,
            total_rides
        )

        # Only consider riders with good match score
        if match_score >= 0.6:
            suitable_riders.append({
                'rider_id': rider_id,
                'rider_data': rider_data,
                'rider_loc': rider_current_loc,
                'rider_dest': rider_dest_loc,
                'rider_name': rider_name,
                'rider_phone': rider_phone,
                'vehicle_type': rider_vehicle_type,
                'rating': rider_rating,
                'total_rides': total_rides,
                'vehicle_number': vehicle_number,
                'vehicle_model': vehicle_model,
                'vehicle_color': vehicle_color,
                'distance_to_pickup': distance_to_pickup,
                'match_score': match_score,
                'priority_level': calculate_priority_level(match_score)
            })
            print(f"  ✅ Suitable rider found: {rider_name} (Score: {match_score:.2f}, Distance: {distance_to_pickup:.2f} km)")

    print(f"  Total {total_riders_checked} riders checked, {len(suitable_riders)} suitable riders found.")

    # Sort riders by match score (highest first)
    suitable_riders.sort(key=lambda x: x['match_score'], reverse=True)
    
    # Create proposals for top suitable riders
    for rider_info in suitable_riders[:5]:  # Limit to top 5 riders
        _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info)

def _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info):
    """Create a proposal document for a specific rider."""
    try:
        # Calculate fare
        passenger_pickup_loc = request_data.get('pickupLocation')
        passenger_dest_loc = request_data.get('destinationLocation')
        
        ride_distance = calculate_distance(
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude,
            passenger_dest_loc.latitude, passenger_dest_loc.longitude
        )
        
        fare_amount = calculate_fare(ride_distance, request_data.get('rideType', 'Standard'))
        
        # Generate OTP
        otp = generate_otp()
        
        # Create proposal document with ALL fields that rider app expects
        proposal_data = {
            'original_request_id': ride_id,
            'riderUid': rider_info['rider_id'],  # This is CRITICAL for rider app filtering
            'riderName': rider_info['rider_name'],
            'riderPhone': rider_info['rider_phone'],
            'riderRating': rider_info['rating'],
            'vehicleNumber': rider_info['vehicle_number'],
            'vehicleModel': rider_info['vehicle_model'],
            'vehicleColor': rider_info['vehicle_color'],
            'vehicleType': rider_info['vehicle_type'],
            'totalRides': rider_info['total_rides'],
            'passengerId': request_data.get('passengerId'),
            'passengerName': request_data.get('passengerName', 'Passenger'),
            'passengerPhone': request_data.get('passengerPhone', 'N/A'),
            'pickupLocation': request_data.get('pickupLocation'),
            'destinationLocation': request_data.get('destinationLocation'),
            'pickupAddress': request_data.get('pickupAddress', 'Pickup Location'),
            'destinationAddress': request_data.get('destinationAddress', 'Destination Location'),
            'fareAmount': fare_amount,
            'paymentMethod': request_data.get('paymentMethod', 'Cash'),
            'rideType': request_data.get('rideType', 'Standard'),
            'passengerRating': request_data.get('passengerRating', 5.0),
            'estimatedDistance': f"{ride_distance:.1f} km",
            'estimatedDuration': f"{(ride_distance / 40 * 60):.0f} min",  # Assuming 40km/h average
            'specialRequests': request_data.get('specialRequests', 'None'),
            'vehiclePreference': request_data.get('vehiclePreference', 'Any'),
            'luggageCount': request_data.get('luggageCount', 0),
            'passengerCount': request_data.get('passengerCount', 1),
            'status': PROPOSAL_PENDING,  # This is now 'pending_acceptance'
            'requestTimestamp': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
            'distanceToPickup': rider_info['distance_to_pickup'] * 1000,  # Convert to meters
            'match_score': rider_info['match_score'],
            'priority_level': rider_info['priority_level'],
            'otp': otp,
            'otpVerified': False,
            # Additional fields that rider app might expect
            'createdAt': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
            'proposalExpiresAt': firestore.SERVER_TIMESTAMP,  # FIXED: No parentheses
        }
        
        # Add proposal to driver_proposals collection
        proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document()
        proposal_ref.set(proposal_data)
        
        print(f"  📨 Proposal sent to {rider_info['rider_name']} (ID: {proposal_ref.id})")
        print(f"     Match Score: {rider_info['match_score']:.2f}, Fare: ₹{fare_amount:.2f}, OTP: {otp}")
        print(f"     Status: {PROPOSAL_PENDING}, Rider UID: {rider_info['rider_id']}")
        
    except Exception as e:
        print(f"[ERROR] Failed to create proposal for rider {rider_info['rider_id']}: {e}")

# --- Snapshot Listeners ---

def on_new_ride_request_snapshot(col_snapshot, changes, read_time):
    """Listener for new ride requests in public_ride_requests."""
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            # Only process new pending requests
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New PENDING ride request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)

def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """Listener for rider responses to proposals and status updates."""
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            previous_data = getattr(change.document, '_previous', {})
            
            current_status = data.get('status')
            previous_status = previous_data.get('status') if previous_data else None
            
            print(f"[PROPOSAL UPDATE] Proposal {change.document.id}: {previous_status} -> {current_status}")
            
            # Handle rider acceptance/rejection
            if (current_status != previous_status and 
                current_status in [PROPOSAL_ACCEPTED, PROPOSAL_REJECTED]):
                
                if current_status == PROPOSAL_ACCEPTED:
                    print(f"[EVENT] Rider accepted proposal: {change.document.id}")
                    # Add proposal ID to data for cleanup
                    data['proposal_id'] = change.document.id
                    finalize_match(db_passenger, db_driver, data)
                elif current_status == PROPOSAL_REJECTED:
                    print(f"[EVENT] Rider rejected proposal: {change.document.id}")
                    # Add proposal ID to data for cleanup
                    data['proposal_id'] = change.document.id
                    handle_rejection(db_passenger, db_driver, data)
            
            # Handle ride status updates from rider app
            elif current_status in [STATUS_ARRIVED_AT_PICKUP, STATUS_PICKED_UP, STATUS_ON_WAY, STATUS_COMPLETED, STATUS_CANCELLED]:
                if current_status != previous_status:
                    print(f"[RIDE STATUS] Ride status changed to: {current_status}")
                    handle_ride_status_update(db_passenger, db_driver, data, current_status)

def on_rider_status_snapshot(col_snapshot, changes, read_time):
    """Listener for rider status changes to track online/offline riders."""
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            rider_id = change.document.id
            status = data.get('status', 'offline')
            location = data.get('currentLocation')
            
            if status == RIDER_AVAILABLE and location:
                print(f"[RIDER STATUS] Rider {rider_id} is available at ({location.latitude:.4f}, {location.longitude:.4f})")
            elif status == RIDER_ON_TRIP:
                print(f"[RIDER STATUS] Rider {rider_id} is on trip")
            elif status == RIDER_OFFLINE:
                print(f"[RIDER STATUS] Rider {rider_id} went offline")

def cleanup_old_proposals():
    """Clean up old proposals that are no longer relevant."""
    try:
        # Delete proposals older than 1 hour
        one_hour_ago = datetime.now() - timedelta(hours=1)
        
        proposals_query = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        old_proposals = proposals_query.get()
        
        deleted_count = 0
        for proposal in old_proposals:
            proposal_data = proposal.to_dict()
            proposal_time = proposal_data.get('requestTimestamp')
            
            # Skip if no timestamp
            if not proposal_time:
                continue
                
            # Convert Firestore timestamp to datetime
            if hasattr(proposal_time, 'timestamp'):
                proposal_datetime = datetime.fromtimestamp(proposal_time.timestamp())
            else:
                continue
                
            if proposal_datetime < one_hour_ago:
                if proposal_data.get('status') in [PROPOSAL_PENDING, PROPOSAL_REJECTED]:
                    proposal.reference.delete()
                    deleted_count += 1
                    print(f"🧹 Cleaned up old proposal: {proposal.id}")
        
        if deleted_count > 0:
            print(f"🧹 Cleaned up {deleted_count} old proposals")
                
    except Exception as e:
        print(f"[CLEANUP ERROR] Failed to clean up old proposals: {e}")

# --- Main Execution ---

if __name__ == '__main__':
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"🚗 RideHail Pro Matching Server Started")
    print(f"📊 Configuration:")
    print(f"   - Max Match Distance: {MAX_MATCH_DISTANCE_KM} km")
    print(f"   - Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km")
    print(f"   - Dynamic Pickup Threshold: {DYNAMIC_PICKUP_THRESHOLD_KM} km")
    print(f"   - Proposal Status: {PROPOSAL_PENDING}")  # Show the corrected status
    print(f"   - Listening on collections: public_ride_requests, driver_proposals, riders")
    
    try:
        # Listener 1: Watch for new ride requests in public_ride_requests
        query_ride_requests = db_passenger.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).where('status', '==', STATUS_PENDING)
        query_ride_requests.on_snapshot(on_new_ride_request_snapshot)
        
        # Listener 2: Watch for rider responses in driver_proposals
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        # Listener 3: Watch for rider status changes
        query_riders = db_driver.collection(RIDER_COLLECTION_NAME)
        query_riders.on_snapshot(on_rider_status_snapshot)
        
        print("-" * 60)
        print("✅ Three listeners started successfully:")
        print("   1. New Ride Requests (public_ride_requests)")
        print("   2. Rider Responses (driver_proposals)") 
        print("   3. Rider Status Changes (riders)")
        print(f"📨 Proposal Status: '{PROPOSAL_PENDING}' (Fixed to match rider app)")
        print("🛠️  Sentinel Errors FIXED: All SERVER_TIMESTAMP calls corrected")
        print("Press Ctrl+C to stop the server.")
        print("-" * 60)

        # Start cleanup scheduler
        import threading
        def schedule_cleanup():
            while True:
                time.sleep(3600)  # Run every hour
                cleanup_old_proposals()
        
        cleanup_thread = threading.Thread(target=schedule_cleanup, daemon=True)
        cleanup_thread.start()
        
        # Keep the main thread alive
        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("\n🛑 RideHail Pro Matching Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)

Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
🚗 RideHail Pro Matching Server Started
📊 Configuration:
   - Max Match Distance: 10.0 km
   - Max Destination Deviation: 5.0 km
   - Dynamic Pickup Threshold: 2.0 km
   - Proposal Status: pending_acceptance
   - Listening on collections: public_ride_requests, driver_proposals, riders
------------------------------------------------------------
✅ Three listeners started successfully:
   1. New Ride Requests (public_ride_requests)
   2. Rider Responses (driver_proposals)
   3. Rider Status Changes (riders)
📨 Proposal Status: 'pending_acceptance' (Fixed to match rider app)
🛠️  Sentinel Errors FIXED: All SERVER_TIMESTAMP calls corrected
Press Ctrl+C to stop the server.
------------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[EVENT] New PENDING ride request detected: dG2WfnbQHmnhE435Kg1I

[MATCHING] Searching for riders for Ride ID: dG2WfnbQHmnhE435Kg1I
Passenger: chandan68gowda | Vehicle Preference: Any
[RIDER STATUS] Rider KHoVWhQqYOZhRRki0VK9 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5432)
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5432)
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5431)


Thread-ConsumeBidirectionalStream caught unexpected exception Timeout of 300.0s exceeded, last exception: 429 Quota exceeded. and will exit.
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\api_core\grpc_helpers.py", line 170, in error_remapped_callable
    return _StreamingResponseIterator(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\api_core\grpc_helpers.py", line 92, in __init__
    self._stored_first_result = next(self._wrapped)
                                ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\grpc\_channel.py", line 543, in __next__
    return self._next()
           ^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\grpc\_channel.py", line 972, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "Quota exceeded."
	debug_err

[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8592, 77.5431)
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8592, 77.5431)
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5431)
[PROPOSAL UPDATE] Proposal aRI2ujS0Q9Ax1QTsu40Q: None -> accepted
[EVENT] Rider accepted proposal: aRI2ujS0Q9Ax1QTsu40Q

[ACCEPTANCE] Finalizing match for Ride gBfyrWzZ81n7QSXr7e9J with Rider a8u9W0osH4bbzvvmPUbCV2pManf2.
  [DB UPDATE] Public ride gBfyrWzZ81n7QSXr7e9J updated to 'accepted'.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
  [DB UPDATE] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 status changed to 'on_trip'.
[

...

[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[CLEANUP ERROR] Failed to clean up old proposals: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8593, 77.5432)
[PROPOSAL UPDATE] Proposal kUXzidaSSDODSswDvPvA: None -> accepted
[EVENT] Rider accepted proposal: kUXzidaSSDODSswDvPvA

[ACCEPTANCE] Finalizing match for Ride H5oM96GNTrDUSpcMDM5v with Rider a8u9W0osH4bbzvvmPUbCV2pManf2.
  [DB UPDATE] Public ride H5oM96GNTrDUSpcMDM5v updated to 'accepted'.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
  [DB UPDATE] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 status changed to 'on_trip'.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline


[FATAL ERROR] Finalization failed for ride H5oM96GNTrDUSpcMDM5v: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8592, 77.5431)
[PROPOSAL UPDATE] Proposal sumD6oz5RS4U9h6OszTj: None -> accepted
[EVENT] Rider accepted proposal: sumD6oz5RS4U9h6OszTj

[ACCEPTANCE] Finalizing match for Ride Ikz7p6eOtlWgggqP22uD with Rider a8u9W0osH4bbzvvmPUbCV2pManf2.
  [DB UPDATE] Public ride Ikz7p6eOtlWgggqP22uD updated to 'accepted'.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
  [DB UPDATE] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 status changed to 'on_trip'.
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is on trip
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 is available at (12.8592, 77.5431)
[RIDER STATUS] Rider a8u9W0osH4bbzvvmPUbCV2pManf2 went offline


[FATAL ERROR] Finalization failed for ride Ikz7p6eOtlWgggqP22uD: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


[CLEANUP ERROR] Failed to clean up old proposals: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


In [ ]:
@firestore_retry()
def update_ride_status(ride_id: str, new_status: str):
    """Manually update a ride's status and sync rider accordingly."""
    ride_ref = db_passenger.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
    ride_doc = ride_ref.get()
    if not ride_doc.exists:
        logger.warning(f"[ManualUpdate] Ride {ride_id} does not exist.")
        return

    ride_data = ride_doc.to_dict()
    rider_id = ride_data.get('riderUid')
    if not rider_id:
        logger.warning(f"[ManualUpdate] Ride {ride_id} has no assigned rider yet.")
        return

    # Update ride status
    ride_ref.update({'status': new_status, 'updatedAt': firestore.SERVER_TIMESTAMP})
    logger.info(f"[ManualUpdate] Ride {ride_id} status set to '{new_status}'.")

    # Update rider status accordingly
    rider_ref = db_driver.collection(RIDER_COLLECTION_NAME).document(rider_id)
    if new_status == STATUS_ARRIVED_AT_PICKUP:
        rider_ref.update({'status': RIDER_ON_TRIP, 'updatedAt': firestore.SERVER_TIMESTAMP})
        logger.info(f"[ManualUpdate] Rider {rider_id} marked as 'on_trip'.")
    elif new_status == STATUS_PICKED_UP:
        rider_ref.update({'status': RIDER_ON_TRIP, 'updatedAt': firestore.SERVER_TIMESTAMP})
        logger.info(f"[ManualUpdate] Rider {rider_id} confirmed picked up passenger.")
    elif new_status == STATUS_COMPLETED:
        rider_ref.update({'status': RIDER_AVAILABLE, 'updatedAt': firestore.SERVER_TIMESTAMP})
        logger.info(f"[ManualUpdate] Rider {rider_id} is now available.")
